<h1><center> TP : NN et CNN avec ```pytorch``` </center></h1>

Note : l'objectif de ce TP est de se familiariser avec 'pytorch' et de comprendre comment implémenter des réseaux de neurones avec Python.

Nous listons d'abord les fonctions de base de pytorch et considérons un exemple très simple pour comprendre comment la descente de gradient peut être implémentée. Ensuite, nous illustrons comment définir l'architecture des réseaux de neurones et l'exécutons sur l'ensemble de données MNIST. Enfin, nous proposons une implémentation de CNN.

En guise de devoir, nous vous proposons d'implémenter la régression logistique sous forme de réseau de neurones et d'ajouter également le dropout dans CNN. 

In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

ModuleNotFoundError: No module named 'tqdm.version'

Pytorch fonctionne avec des tenseurs au lieu de tableaux numpy. Presque tout ce que vous pouvez faire avec les tableaux numpy peut être accompli avec des tenseurs Pytorch. 

In [10]:
x = torch.rand(3, 3) # random tensor of size 3 by 3
print(x)

tensor([[0.3883, 0.9711, 0.1386],
        [0.9873, 0.8204, 0.2703],
        [0.7844, 0.3789, 0.7307]])


In [11]:
# We can operate with pytorch tensors pretty much in the same manner as with numpy arrays
x = torch.ones(3,3)
y = torch.ones(3,3) * 4
z = x + y
print(f'This is the result of:\n {x}\n +\n {y} \n = \n {z}')

This is the result of:
 tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
 +
 tensor([[4., 4., 4.],
        [4., 4., 4.],
        [4., 4., 4.]]) 
 = 
 tensor([[5., 5., 5.],
        [5., 5., 5.],
        [5., 5., 5.]])


In [13]:
# again we can operate with tensor indexing as if it was a numpy one

x = torch.ones(3,3) * 5
y = x[-1, :2]
print(f'From \n {x} we can look at the last column and 2 rows \n {y}')



From 
 tensor([[5., 5., 5.],
        [5., 5., 5.],
        [5., 5., 5.]]) we can look at the last column and 2 rows 
 tensor([5., 5.])


**Exemple**

Comme vous le savez, de nombreux algorithmes de ML peuvent être considérés comme des problèmes d'optimisation.

Considérons un exemple jouet : imaginez que nos données soient constituées d'une part de $x = (1, \ldots, 1)^\top \in \mathbb{R}^{5}$ qui est un vecteur composé que de $1$ et d'autre part de l'étiquette $y = 1$. On aimerait trouver un vecteur de poids $w \in \mathbb{R}^{5}$ tel que la fonction de perte $L(w) = (y - x^\top w)^2$ soit minimisée.

Bien sûr, il s'agit d'une régression moindre carré simple sur une seule observation $(x, y)$ et nous pouvons calculer le résultat analytiquement. Mais c'est un bon exemple pour comprendre ce que Pytorch a à offrir.

Si nous sommes trop paresseux pour calculer l'expression analytique, nous pouvons exécuter la descente de gradient, qui commence à partir de $w_0 = (0, \ldots, 0)^\top$ et continue de la façon suivante :

$$w_k = w_{k - 1} - \eta \nabla L(w_{k - 1}).$$

Donc, la seule chose que nous devons savoir est le gradient de la fonction de perte $L$ évalué au point $w_{k - 1}$.
Voici comment cela se fait en pytorch. 

In [14]:
# Input data
y = torch.ones(1, 1)
x = torch.ones(1, 5)

# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these tensors during the backward pass.
w = torch.zeros(5, 1, requires_grad=True) # setting w_0 = (0, ..., 0)^T

y_pred = x.mm(w) # inner product of w and x 

loss = (y - y_pred).pow(2) # squared loss


# Use autograd to compute the backward pass. This call will compute the
# gradient of loss with respect to all tensors with requires_grad=True.
# After this call w.grad will be a tensor holding the gradient
# of the loss with respect to w.
loss.backward()

print(w.grad) # Print the gradient


tensor([[-2.],
        [-2.],
        [-2.],
        [-2.],
        [-2.]])


**Question.** En supposant que $w_0 = (0, \ldots, 0)^\top$ calculez sur le papier $\nabla L(w_0)$. N'incluez pas la réponse à cette question dans le rapport. Assurez-vous simplement de comprendre ce qui se passe ici.

Une fois que vous vous êtes assuré que ```w.grad``` stocke bien la valeur de $\nabla L(w_0)$. Nous pouvons implémenter l'algorithme de descente de gradient avec seulement quelques lignes de code ! 

In [15]:
# Input data
y = torch.ones(1, 1)
x = torch.ones(1, 5)

w = torch.zeros(5, 1, requires_grad=True) # Initialization: w_0 = (0, ..., 0)^T

lr = 0.01 # Learning rate a.k.a. the step size
max_iter = 150

for k in range(max_iter):
    loss = (y - x.mm(w)).pow(2) # forward pass
    
        
    loss.backward() # the backward pass
    
    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        w -= lr * w.grad # gradient step
        w.grad.zero_() # after performing operation with gradient we need to erase it
    
    if k % 10 == 9 or k == 0:
        print(f'Iteration {k + 1}/{max_iter}, Current loss: {loss.item()}')
#         print(y.grad)
        
print(f'Final result: {w}')

Iteration 1/150, Current loss: 1.0
Iteration 10/150, Current loss: 0.150094673037529
Iteration 20/150, Current loss: 0.018248017877340317
Iteration 30/150, Current loss: 0.002218528650701046
Iteration 40/150, Current loss: 0.00026972233899869025
Iteration 50/150, Current loss: 3.279230440966785e-05
Iteration 60/150, Current loss: 3.986556748714065e-06
Iteration 70/150, Current loss: 4.846697265747935e-07
Iteration 80/150, Current loss: 5.8908199207508005e-08
Iteration 90/150, Current loss: 7.173785121494802e-09
Iteration 100/150, Current loss: 8.74024408403784e-10
Iteration 110/150, Current loss: 1.0756195933936397e-10
Iteration 120/150, Current loss: 1.3219647598816664e-11
Iteration 130/150, Current loss: 1.566746732351021e-12
Iteration 140/150, Current loss: 1.7408297026122455e-13
Iteration 150/150, Current loss: 1.2789769243681803e-13
Final result: tensor([[0.2000],
        [0.2000],
        [0.2000],
        [0.2000],
        [0.2000]], requires_grad=True)


**Question :** Résoudre le problème $\min_{w \in \mathbb{R}^5}\, (1 - x^\top w)^2$ avec $x = (1, \ldots, 1) ^\top \in \mathbb{R}^5$ analytiquement et comparer au résultat de la descente de gradient.
 
**Question :** Expliquez la connexion de ```loss.backward()``` et la rétropropagation pour les réseaux de neurones feedforward. 

# Multi layer perceptron

Ci-dessous, nous allons construire notre réseau neuronal. Rappelons que MNIST est composé d'images de taille $28 \times 28$, donc la dimension de l'entrée est $784$. Nous avons $10$ classes, donc la dimension de la sortie est $10$.

Entre les deux, nous insérerons $2$ couches cachées et utiliserons ReLU comme non-linéarité (fonction d'activation).
La première couche cachée est composée de neurones à $128$ et la seconde de neurones à $64$.

Nous n'utiliserons pas de GPU ni ne considérerons des réseaux de neurones compliqués dans ce TP. Le but est de vous initier aux bases sans entrer dans des architectures trop compliquées.

In [16]:
class SimpleFeedForward(nn.Module):
    def __init__(self, input_size=784, hidden_sizes=[128, 64],
                 output_size=10):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, hidden_sizes[0]), 
            nn.ReLU(),
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.ReLU(),
            nn.Linear(hidden_sizes[1], output_size)
        )
             
    def forward(self, x):
        x = x.reshape(-1, input_size)
        x = self.classifier(x)
        return x

Une fois que nous avons défini notre réseau de neurones, nous devons l'entraîner.
L'entraînement va être effectuée via la descente de gradient stochastique évaluée sur un mini batch des données.
C'est-à-dire qu'au stade avancé, nous n'utiliserons pas un seul point de données mais plusieurs. Dans cet exemple, nous fixons la taille du mini batch à $32$.

En fait, la taille du mini batch, la valeur de la learning rate, les tailles des couches cachées sont tous considérés comme des hyperparamètres qui peuvent être finement réglés (certaines personnes règlent même des graines aléatoires, ce qui est absolument ridicule). Nous ne parlerons pas du réglage des hyperparamètres dans ce TP, pour en savoir plus, consultez https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html .


**Important :** Nous ne vous demandons pas d'effectuer un réglage compliqué des hyperparamètres. Cette partie n'est pas l'object du TP. Cependant, il est important que vous puissiez clairement écrire l'architecture du réseau de neurones que vous considérez. 

In [17]:
# Training consists of gradient steps over mini batch of data
def train(model, trainloader, loss, optimizer, epoch, num_epochs):
    # We enter train mode. This is useless for the linear model
    # but is important for layers such as dropout, batchnorm, ...
    model.train()
    
    loop = tqdm(trainloader)
    loop.set_description(f'Training Epoch [{epoch + 1}/{num_epochs}]')
    
    # We iterate over the mini batches of our data
    for inputs, targets in loop:
        
        # Erase any previously stored gradient
        optimizer.zero_grad()
        
        
        outputs = model(inputs) # Forwards stage (prediction with current weights)
        my_loss = criterion(outputs, targets) # loss evaluation
        
        my_loss.backward() # Back propagation (evaluate gradients) 
        
        
        # Making gradient step on the batch (this function takes care of the gradient step for us)
        optimizer.step() 
        
def validation(model, valloader, loss):
    # Do not compute gradient, since we do not need it for validation step
    with torch.no_grad():
        # We enter evaluation mode.
        model.eval()
        
        total = 0 # keep track of currently used samples
        running_loss = 0.0 # accumulated loss without averagind
        accuracy = 0.0 # accumulated accuracy without averagind (number of correct predictions)
        
        loop = tqdm(valloader) # This is for the progress bar
        loop.set_description('Validation in progress')
        
        
        # We again iterate over the batches of validation data. batch_size does not play any role here
        for inputs, targets in loop:
            # Run samples through our net
            outputs = model(inputs)

            # Total number of used samples
            total += inputs.shape[0]

            # Multiply loss by the batch size to erase averagind on the batch
            running_loss += inputs.shape[0] * loss(outputs, targets).item()
            
            # how many correct predictions
            accuracy += (outputs.argmax(dim=1) == targets).sum().item()
            
            # set nice progress meassage
            loop.set_postfix(val_loss=(running_loss / total), val_acc=(accuracy / total))
        return running_loss / total, accuracy / total

Nous utilisons à nouveau le jeu de données MNIST. Cette fois, nous utiliserons le split train/test officiel ! 

In [18]:
# We download the oficial MNIST train set
all_train = datasets.MNIST('data/',
                           download=True,
                           train=True,
                           transform=transforms.ToTensor())

# We split the whole train set in two parts:
# the one that we actually use for training
# and the one that we use for validation
batch_size = 32 # size of the mini batch
num_train = int(0.8 * len(all_train))

trainset, valset = torch.utils.data.random_split(all_train, [num_train, len(all_train) - num_train])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100.0%


Extracting data/MNIST\raw\train-images-idx3-ubyte.gz to data/MNIST\raw



102.8%


Extracting data/MNIST\raw\train-labels-idx1-ubyte.gz to data/MNIST\raw



100.0%


Extracting data/MNIST\raw\t10k-images-idx3-ubyte.gz to data/MNIST\raw



112.7%
C:\Users\berni\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)


Extracting data/MNIST\raw\t10k-labels-idx1-ubyte.gz to data/MNIST\raw

Processing...
Done!


In [ ]:
# we can iterate over trainloader in the following way
for inputs, targets in trainloader:
    print(f'Dimensions of the inputs are {inputs.shape}')
    plt.imshow(inputs[0][0], cmap='gray', interpolation='none')
    plt.show()
    print(f'The number on the image is: {targets[0]}')
    break

La forme des ```entrées``` est $(32, 1, 28, 28)$. La première dimension indique la taille du mini batch et est contrôlée par le paramètre ```batch_size```, les deux derniers paramètres sont les dimensions 2D de l'image et sont égaux à $28 \times 28$ dans le cas des données MNIST. Le $1$ restant dans la deuxième dimension reflète essentiellement le fait que les images sont en noir et blanc. Par exemple, si MNIST était une base colorée (il existe en fait des variantes de MNIST colorée), alors nous aurions besoin de $3$ (en cas de RVB) de couleurs pour représenter une image, donc $1$ serait remplacé par $3$.

**Question :** Exécutez le bloc ci-dessus plusieurs fois. Trace-t-il le même nombre tout le temps ? Si non, pourquoi ? 

In [ ]:
# Net + training parameters
num_epochs = 2 # how many passes over the whole train data
input_size = 784 # flattened size of the image
hidden_sizes = [128, 64] # sizes of hidden layers
output_size = 10 # how many labels we have
lr = 0.001 # learning rate
momentum = 0.9 # momentum

In [ ]:
# initializing our model/loss/optimizer
net = SimpleFeedForward(input_size, hidden_sizes, output_size) # Our neural net
criterion = nn.CrossEntropyLoss() # Loss function to be optimized
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum) # Optimization algorithm

In [ ]:
# num_epochs indicates the number of passes over the data
for epoch in range(num_epochs):
    
    # makes one pass over the train data and updates weights
    train(net, trainloader, criterion, optimizer, epoch, num_epochs)

    # makes one pass over validation data and provides validation statistics
    val_loss, val_acc = validation(net, valloader, criterion)


In [ ]:
# Let us evaluate our net on the test set that we have never seen!
testset = datasets.MNIST('data/',
                         download=True,
                         train=False,
                         transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

test_loss, test_acc = validation(net, testloader, criterion)
print(f'Test accuracy: {test_acc} | Test loss: {test_loss}')

# Problème 1: Régression Logistique via pytorch

En utilisant le code ci-dessus comme exemple, implémentez la régression logistique multinomiale et entraînez-la sur les mêmes données.
Vous devez pouvoir :
1. Expliquer l'dée générale sur la façon d'implémenter la régression logistique avec pytorch
2. fournir l'exactitude de la classification sur les données de test 

# Elements de CNN: ```nn.Conv2d``` et ```MaxPool2d```

Lisez ceci avant de commencer : https://ttic.uchicago.edu/~shubhendu/Pages/Files/Lecture7_flat.pdf

**Comprendre les layers convolutifs dans pytorch**

Une fois que nous avons instancié ```nn.Conv2d(1, 1, kernel_size=2, stride=[1, 1], padding=0)``` il a un paramètre ```weight``` qui décrit précisément le noyau utilisé pour notre convolution. Au début, il est initialisé de manière aléatoire, et notre objectif est d'entraîner éventuellement ses poids (comme d'habitude via la rétropropagation !).
Avant de construire notre premier CNN, examinons le noyau et ce qu'il fait. 

In [ ]:
# 1 input channel (first 1 in nn.Conv2d)
# 1 output channel (second 1 in nn.Conv2d)
# 2x2 kernel (kernel_size=2)
# the kernel slides by one step in (x, y) direction (stride=[1, 1])
# we do not augment the picture with white borders (padding=0)
conv = nn.Conv2d(1, 1, kernel_size=2, stride=[1, 1], padding=0) 
# Get kernel value.
weight = conv.weight.data.numpy()

**Visualisation.** Nous allons tracer l'image initiale, le noyau et l'image résultante. Afin de comprendre ce qui se passe, l'image résultante sera calculée de deux manières. Tout d'abord, il sera calculé en utilisant ```conv1(image)```. Deuxièmement, nous appliquerons manuellement le noyau coulissant à chaque fenêtre de taille $2\times 2$. 

In [ ]:
# take one image
image, _ = next(iter(trainloader))


fig, axs = plt.subplots(1, 4)
fig.tight_layout()
fig.suptitle('Convolution')

# plot the image
axs[0].imshow(image[0][0], cmap='gray', interpolation='none')
axs[0].set_title('Original image')

# plot the kernel
axs[1].imshow(weight[0][0], cmap='gray', interpolation='none')
axs[1].set_title('2x2 kernel')

# plot resulting image
axs[2].imshow(conv(image)[0][0].detach().numpy(), cmap='gray', interpolation='none')
axs[2].set_title('Resulting image')

# Making the same by hands
# IMPORTANT: we strongly suggest to understand the below code
np_image = image[0][0].data.numpy() # get numpy image
image_convolved = np.zeros((27, 27)) # here we store our result
for i in range(27):
    for j in range(27):
        image_convolved[i, j] = np.sum(np_image[i:i+2, j:j+2] * weight) # apply the kernel for each 2x2 window
        
axs[3].imshow(image_convolved, cmap='gray', interpolation='none')
axs[3].set_title('By hand')

**Problème.** Fournir une implémentation « à la main » du noyau suivant 

In [ ]:
# 1 input channel (first 1 in nn.Conv2d)
# 1 output channel (second 1 in nn.Conv2d)
# 4x4 kernel (kernel_size=4)
# the kernel slides by 3 step in (x, y) direction (stride=[4, 4])
# we do not augment the picture with white borders (padding=0)
conv = nn.Conv2d(1, 1, kernel_size=4, stride=[4, 4], padding=0) 
# Get kernel value.
weight = conv.weight.data.numpy()

# take one image
image, _ = next(iter(trainloader))


fig, axs = plt.subplots(1, 4)
fig.tight_layout()
fig.suptitle('Convolution')

# plot the image
axs[0].imshow(image[0][0], cmap='gray', interpolation='none')
axs[0].set_title('Original image')

# plot the kernel
axs[1].imshow(weight[0][0], cmap='gray', interpolation='none')
axs[1].set_title('4x4 kernel')

# plot resulting image
axs[2].imshow(conv(image)[0][0].detach().numpy(), cmap='gray', interpolation='none')
axs[2].set_title('Resulting image')

# Making the same by hands
# PROBLEM: FILL IN THIS PART. 
np_image = image[0][0].data.numpy() # get numpy image

**Comprendre la couche de pooling (mise en commun) dans Pytorch** 

Le max pooling est ce qui est souvent utilisé en pratique, il revient à ne choisir que la plus grande valeur d'un pixel dans une fenêtre donnée. Dans pytorch, cela se fait via ```MaxPool2d(kernel_size=k, stride=s)```, qui a deux paramètres : la taille du noyau et la 'stride'. Notez qu'il n'y a pas de poids à apprendre ici, donc cette couche est simplement fixe. 

In [ ]:
# kernel_size -- size of the max pool window
pool = nn.MaxPool2d(kernel_size=4, stride=[4,4])

fig, axs = plt.subplots(1, 3)
fig.tight_layout()
fig.suptitle('Pooling')

# plot the image
axs[0].imshow(image[0][0], cmap='gray', interpolation='none')
axs[0].set_title('Original image')


# plot resulting image
axs[1].imshow(pool(image)[0][0].detach().numpy(), cmap='gray', interpolation='none')
axs[1].set_title('Resulting image')

# Making the same by hands
# IMPORTANT: we strongly suggest to understand the below code
np_image = image[0][0].data.numpy() # get numpy image
image_pooled = np.zeros((7, 7)) # here we store our result
for i in range(0, 27, 4):
    for j in range(0, 27, 4):
        image_pooled[int(i / 4), int(j / 4)] = np.max(np_image[i:i+4, j:j+4]) # max pooling
        
axs[2].imshow(image_pooled, cmap='gray', interpolation='none')
axs[2].set_title('By hand')

# Construire un ConvNet simple

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=5, stride=[1, 1], padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.classifier = nn.Sequential(
            nn.Linear(14 * 14 * 8, 500),
            nn.ReLU(),
            nn.Linear(500, 10),
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = out.reshape(out.size(0), -1)
        out = self.classifier(out)
        return out

Notez que la première couche est ```nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2)```, les paramètres ici sont choisis de telle sorte que la taille de chaque canal de sortie reste de taille $28 \times 28 $. En effet, en fixant ```padding = 2``` nous avons augmenté notre image initiale à $32 \times 32$, puis nous glissons un noyau de taille $5 \times 5$ par $1$ dans les deux directions $(x, y)$ qui résultat en une image de sortie $28 \times 28$ (et $8$ canaux).

En général, la formule pour les images carrées et les noyaux carrés est
$$
    S_{out} = \frac{S_{in} - S_{noyau} + 2S_{padding}}{S_{stride}} + 1
$$

Dans notre cas c'est

$$
    S_{out} = \frac{28 - 5 + 4}{1} + 1 = 28
$$

Ensuite, la sortie de ```nn.Conv2d(1, 8, kernel_size=5, stride=1, padding=2)``` va dans ```nn.ReLU()``` notre non-linéarité préférée et finalement dans la couche de pooling ```nn.MaxPool2d(kernel_size=2, stride=2)```.
Le ```nn.ReLU()``` n'affecte pas la taille, donc ```nn.MaxPool2d(kernel_size=2, stride=2)``` reçoit $8$ canaux d'images de taille $28 \times 28$  calculés comme ci-dessus.

```nn.MaxPool2d(kernel_size=2, stride=2)``` sera appliqué à chaque canal, avec ```kernel_size=2, stride=2``` signifiant que la sortie aura toujours $8$ canaux mais les images seront divisées en deux dans les deux directions $(x, y)$. Par conséquent, la sortie de ```nn.MaxPool2d(kernel_size=2, stride=2)``` a $8$ canaux avec $14 \times 14$ images.

Après tout cela, nous allons aplatir nos fonctionnalités et les mettre dans un simple ```nn.Linear(14 * 14 * 8, 500)```, où la taille d'entrée est précisément la taille de sortie de ```nn.MaxPool2d( kernel_size=2, stride=2)```, et $500$ représentent la taille de sortie de cette couche linéaire.
Enfin, nous appliquons notre non-linéarité préférée à ```nn.Linear(14 * 14 * 8, 500)``` suivie d'une couche linéaire entièrement connectée ```nn.Linear(500, 10)``` pour correspondre à la dimension de cours à $10$. 

In [ ]:
net = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

for epoch in range(num_epochs):
    
    # makes one pass over the train data and updates weights
    train(net, trainloader, criterion, optimizer, epoch, num_epochs)

    # makes one pass over validation data and provides validation statistics
    val_loss, val_acc = validation(net, valloader, criterion)

In [ ]:
test_loss, test_acc = validation(net, testloader, criterion)
print(f'Test accuracy: {test_acc} | Test loss: {test_loss}')

Comme vous le voyez, le résultat ici est bien meilleur que dans le simple perceptron multilayer. Mais notez que nous avons en fait entraîné beaucoup plus de paramètres ici et, du moins sur mon ordinateur, cela prend beaucoup plus de temps.

Ici vous pouvez voir le résumé des résultats actuels de l'état de l'art sur MNIST : https://www.kaggle.com/c/digit-recognizer/discussion/61480

Comme vous le voyez, notre score bat à peine une forêt aléatoire soigneusement construite ou **kNN** ! Pour obtenir $0,01 $ supplémentaire, il faut beaucoup plus de réglages, ce qui n'est bien sûr pas le but ici. 

# Problème 2 : Dropout

Modifiez le code pour ConvNet et insérez la couche Dropout (où vous le souhaitez).

Incluez dans votre rapport :
1. Description générale du dropout
2. Description générale de votre architecture

# Conclusion

Après la réussite de ce TP, nous attendons de vous que vous soyez capable de comprendre les architectures de NN, CNN.
Par exemple, jetez un oeil au célèbre AlexNet https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py et voyez si vous pouvez comprendre son architecture.